In [44]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [49]:
url = "https://api.worldbank.org/v2/country/"
format = "json"
per_page = 50
def api_request_all_pages(url,date_start=None,date_end=None):
    result = []
    page = 1
    while True:
        params = {
            "source":2,
            "format": format,
            "per_page": per_page,
            "page": page
           
        }
        if date_start != None and date_end != None:
            params["date"]=f"{str(date_start)}:{str(date_end)}"
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        payload = response.json()
        total_pages = payload[0]["pages"]
        result += payload[1]

        if page < total_pages:
            page += 1
        else:
            break
    return result

countrys=["ESP","VEN","USA","ITA","ARG"]
url_with_countrys=url+";".join(countrys)
# payload=api_request_all_pages(url_with_countrys)
# payload

In [ ]:
# Población total
# PIB per cápita (USD actuales) 
# Tasa de desempleo
# Esperanza de vida al nacer (años)   
indicators=["SP.POP.TOTL","NY.GDP.PCAP.CD","SL.UEM.TOTL.ZS","SP.DYN.LE00.IN"]
date_start=2010
date_end=2024
final_url = url_with_countrys+"/indicator/"+";".join(indicators)

countries_with_indicator = api_request_all_pages(final_url,date_start,date_end) 


[{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2024',
  'value': 45696159,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2023',
  'value': 45538401,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2022',
  'value': 45407904,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2021',
  'value': 45312281,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, t

In [ ]:
def clean_indicator_df(df_row):
    df_row["indicator"]=df_row["indicator"]["id"]
    df_row["country"]=df_row["country"]["value"]
    df_row["date"]=int(df_row["date"])
    df_row["value"]=round(float(df_row["value"]),2)
    return df_row
    
df_countries_with_indicator=pd.DataFrame(countries_with_indicator)
df_countries_with_indicator=df_countries_with_indicator[["indicator","country","countryiso3code","date","value"]].dropna(axis=0)
df_countries_with_indicator=df_countries_with_indicator.apply(clean_indicator_df,axis=1)
df_countries_with_indicator.columns=["Indicador","Pais","Pais(Iso3)","Año","Valor"]
df_countries_with_indicator

,Indicador,Pais,Pais(Iso3),Año,Valor
0,SP.POP.TOTL,Argentina,ARG,2024,45696159.00
1,SP.POP.TOTL,Argentina,ARG,2023,45538401.00
2,SP.POP.TOTL,Argentina,ARG,2022,45407904.00
3,SP.POP.TOTL,Argentina,ARG,2021,45312281.00
4,SP.POP.TOTL,Argentina,ARG,2020,45191965.00
...,...,...,...,...,...
295,SP.DYN.LE00.IN,"Venezuela, RB",VEN,2014,72.84
296,SP.DYN.LE00.IN,"Venezuela, RB",VEN,2013,73.08
297,SP.DYN.LE00.IN,"Venezuela, RB",VEN,2012,73.09
298,SP.DYN.LE00.IN,"Venezuela, RB",VEN,2011,73.09


In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(
    data=df_countries_with_indicator,
    x="PIB_per_capita",
    y="Esperanza De Vida",
    hue="country",
    size="unemployment_rate",
    sizes=(50, 300),
    palette="viridis"
)

plt.title("PIB per cápita vs Esperanza de vida")
plt.xlabel("PIB per cápita (USD actuales)")
plt.ylabel("Esperanza de vida (años)")
plt.tight_layout()
plt.show()
